## imports

In [1]:
from datetime import datetime
from pyspark.sql import functions as F
from pyspark.sql.types import StructType,StructField, StringType,\
                              DateType, IntegerType, DoubleType, LongType,\
                              BooleanType
from pyspark.sql.functions import col, explode, array, struct, expr, sum, lit

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1609189712855_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## define bucket

In [2]:
input_bucket = 's3://dh-ud-dend-capstone/stage0_raw/gdelt'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Gdelt data

In [3]:
schema = StructType() \
            .add("GLOBALEVENTID",LongType(),True) \
            .add("SQLDATE",StringType(),True) \
            .add("MonthYear",IntegerType(),True) \
            .add("Year",IntegerType(),True) \
            .add("FractionDate",StringType(),True) \
            .add("Actor1Code",StringType(),True) \
            .add("Actor1Name",StringType(),True) \
            .add("Actor1CountryCode",StringType(),True) \
            .add("Actor1KnownGroupCode",StringType(),True) \
            .add("Actor1EthnicCode",StringType(),True) \
            .add("Actor1Religion1Code",StringType(),True) \
            .add("Actor1Religion2Code",StringType(),True) \
            .add("Actor1Type1Code",StringType(),True) \
            .add("Actor1Type2Code",StringType(),True) \
            .add("Actor1Type3Code",StringType(),True) \
            .add("Actor2Code",StringType(),True) \
            .add("Actor2Name",StringType(),True) \
            .add("Actor2CountryCode",StringType(),True) \
            .add("Actor2KnownGroupCode",StringType(),True) \
            .add("Actor2EthnicCode",StringType(),True) \
            .add("Actor2Religion1Code",StringType(),True) \
            .add("Actor2Religion2Code",StringType(),True) \
            .add("Actor2Type1Code",StringType(),True) \
            .add("Actor2Type2Code",StringType(),True) \
            .add("Actor2Type3Code",StringType(),True) \
            .add("IsRootEvent",StringType(),True) \
            .add("EventCode",StringType(),True) \
            .add("EventBaseCode",StringType(),True) \
            .add("EventRootCode",StringType(),True) \
            .add("QuadClass",StringType(),True) \
            .add("GoldsteinScale",StringType(),True) \
            .add("NumMentions",IntegerType(),True) \
            .add("NumSources",IntegerType(),True) \
            .add("NumArticles",IntegerType(),True) \
            .add("AvgTone",DoubleType(),True) \
            .add("Actor1Geo_Type",StringType(),True) \
            .add("Actor1Geo_FullName",StringType(),True) \
            .add("Actor1Geo_CountryCode",StringType(),True) \
            .add("Actor1Geo_ADM1Code",StringType(),True) \
            .add("Actor1Geo_Lat",StringType(),True) \
            .add("Actor1Geo_Long",StringType(),True) \
            .add("Actor1Geo_FeatureID",StringType(),True) \
            .add("Actor2Geo_Type",StringType(),True) \
            .add("Actor2Geo_FullName",StringType(),True) \
            .add("Actor2Geo_CountryCode",StringType(),True) \
            .add("Actor2Geo_ADM1Code",StringType(),True) \
            .add("Actor2Geo_Lat",StringType(),True) \
            .add("Actor2Geo_Long",StringType(),True) \
            .add("Actor2Geo_FeatureID",StringType(),True) \
            .add("ActionGeo_Type",StringType(),True) \
            .add("ActionGeo_FullName",StringType(),True) \
            .add("ActionGeo_CountryCode",StringType(),True) \
            .add("ActionGeo_ADM1Code",StringType(),True) \
            .add("ActionGeo_Lat",StringType(),True) \
            .add("ActionGeo_Long",StringType(),True) \
            .add("ActionGeo_FeatureID",StringType(),True) \
            .add("DATEADDED",StringType(),True) \
            .add("SOURCEURL",StringType(),True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
gdelt_file = '/20201215.export.CSV'
df_gdelt = spark.read.option("delimiter", "\t").csv(input_bucket + gdelt_file,header=False,schema=schema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
datefunc = F.udf(lambda x: datetime.strptime(x, '%Y%m%d'), DateType())
df_gdelt = df_gdelt.withColumn('date', datefunc(col('SQLDATE')))
df_gdelt = df_gdelt.withColumn("month",F.month("date"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df_gdelt_reduced = df_gdelt.select("GLOBALEVENTID",\
                                    "date",\
                                    "month",\
                                    "Year",\
                                    "Actor1Code",\
                                    "Actor1Name",\
                                    "Actor1CountryCode",\
                                    "Actor2Code",\
                                    "Actor2Name",\
                                    "Actor2CountryCode",\
                                    "IsRootEvent",\
                                    "EventCode",\
                                    "EventBaseCode",\
                                    "EventRootCode",\
                                    "QuadClass",\
                                    "GoldsteinScale",\
                                    "NumMentions",\
                                    "NumSources",\
                                    "NumArticles",\
                                    "AvgTone",\
                                    "ActionGeo_Type",\
                                    "ActionGeo_CountryCode",\
                                    "ActionGeo_ADM1Code",\
                                    "SOURCEURL")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
def containsCovidContent(url):
    if url is None:
        return False
    if 'covid' in url.lower():
        return True
    if 'corona' in url.lower():
        return True
    if 'virus' in url.lower():
        return True
    
    return False

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
covidFunc = F.udf(lambda x: containsCovidContent(x), BooleanType())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
df_gdelt_reduced = df_gdelt_reduced.withColumn('covid', covidFunc(col('SOURCEURL')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
df_gdelt_reduced = df_gdelt_reduced.withColumnRenamed("Year","year")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
df_gdelt_reduced.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- GLOBALEVENTID: long (nullable = true)
 |-- date: date (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- Actor1Code: string (nullable = true)
 |-- Actor1Name: string (nullable = true)
 |-- Actor1CountryCode: string (nullable = true)
 |-- Actor2Code: string (nullable = true)
 |-- Actor2Name: string (nullable = true)
 |-- Actor2CountryCode: string (nullable = true)
 |-- IsRootEvent: string (nullable = true)
 |-- EventCode: string (nullable = true)
 |-- EventBaseCode: string (nullable = true)
 |-- EventRootCode: string (nullable = true)
 |-- QuadClass: string (nullable = true)
 |-- GoldsteinScale: string (nullable = true)
 |-- NumMentions: integer (nullable = true)
 |-- NumSources: integer (nullable = true)
 |-- NumArticles: integer (nullable = true)
 |-- AvgTone: double (nullable = true)
 |-- ActionGeo_Type: string (nullable = true)
 |-- ActionGeo_CountryCode: string (nullable = true)
 |-- ActionGeo_ADM1Code: string (nullable = tru

In [12]:
#source: https://www.geodatasource.com/resources/tutorials/international-country-code-fips-versus-iso-3166/
raw_data = 's3://dh-ud-dend-capstone/stage0_raw'
mapping_file = '/mapping_fips_to_iso3166.csv'
df_mapping = spark.read.option("delimiter", "\t").csv(raw_data + mapping_file,header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
df_mapping = df_mapping.select('FIPS 10-4 ', 'ISO 3166-1') 
df_mapping = df_mapping.withColumnRenamed("FIPS 10-4 ","ActionGeo_CountryCode") \
                       .withColumnRenamed("ISO 3166-1","country_code")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
stripFunc = F.udf(lambda x: x.strip(),StringType())
df_mapping = df_mapping.withColumn('ActionGeo_CountryCode', stripFunc(col('ActionGeo_CountryCode'))) \
                       .withColumn('country_code', stripFunc(col('country_code')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
df_gdelt_reduced = df_gdelt_reduced.join(df_mapping, on=['ActionGeo_CountryCode'], how='left')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
df_gdelt_reduced.write.mode('overwrite').partitionBy("country_code").parquet("s3://dh-ud-dend-capstone/stage1_prepared_and_partitioned/gdelt/gdelt.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…